In [ ]:
# https://www.jstatsoft.org/article/view/v035c02/v35c02.pdf

In [1]:
import pandas as pd
import pyper as pr

In [2]:
# CREATE A R INSTANCE WITH PYPER:    
r = pr.R()

In [4]:
import pandas as pd
rnb = pd.read_csv('rnb1015_2 AllVars.csv')
rnb.head()
rnb_AMR = rnb[(rnb.region == 'AMR') & (rnb.marketing == 'NotPaid')]
rnb_AMR = rnb_AMR.reset_index().drop(columns=['index'])

In [5]:
rnb_AMR.head()

,date,region,marketing,visits,br,inq,gb,cb,nb,ss,ts,listings
0,2016-01-01,AMR,NotPaid,1256288,6043,66022,6071,571,5500,485968.08,144695.8942,10242216
1,2016-01-02,AMR,NotPaid,1346779,6789,75141,7194,681,6513,554396.44,176412.8379,10236960
2,2016-01-03,AMR,NotPaid,1563675,8132,93361,8307,705,7602,532272.17,205835.1187,10239648
3,2016-01-04,AMR,NotPaid,1755009,9101,101760,9982,1111,8871,1030987.00,249086.6228,10233168
4,2016-01-05,AMR,NotPaid,1761816,9876,108097,11051,1223,9828,1177099.90,271275.1296,10255176


In [6]:
# PASS DATA FROM PYTHON TO R (DF gets stored as rDF)
r.assign("rDF", rnb_AMR["cb"]) 

In [7]:
print (r("summary(rDF)"))

try({summary(rDF)})
       cb      
 Min.   : 239  
 1st Qu.:1038  
 Median :1359  
 Mean   :1391  
 3rd Qu.:1701  
 Max.   :5829  



In [8]:
print (r("str(rDF)"))

try({str(rDF)})
'data.frame':	1019 obs. of  1 variable:
 $ cb: int  571 681 705 1111 1223 1249 1123 1117 868 937 ...



In [9]:
# load R package
r("library(forecast)")

'try({library(forecast)})\n'

In [ ]:
## R snippet to (1) find arima params, (2) fit a arima model, (3) forecast, (4) output a csv
'''
## Auto.Arima
rnb_arima <- auto.arima(DF[,9])  # ALWAYS INPUT RAW DATA THAT IS 'NOT' A ts() object
arimaorder(rnb_arima)
## STLM - apply Auto.Arima model to a ts object
# Make the 'y' variables to a ts object
rnb <- ts(DF$y, start=2016,freq=365)
fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))
fit_arima_fc <- forecast(fit, h=442)
write.csv(fit_arima_fc, "6_rnb_arima_pred_fc.csv")
'''

In [11]:
r("rnb_arima <- auto.arima(rDF$cb, seasonal = T)")

'try({rnb_arima <- auto.arima(rDF$cb, seasonal = T)})\n'

In [12]:
print(r("arimaorder(rnb_arima)"))

try({arimaorder(rnb_arima)})
p d q 
3 1 2 



In [13]:
# convert data frame into a ts object
r("rnb <- ts(rDF$cb, start=2016,freq=365)")

'try({rnb <- ts(rDF$cb, start=2016,freq=365)})\n'

In [14]:
r("fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))")

'try({fit <- stlm(rnb, modelfunction=Arima, order=arimaorder(rnb_arima))})\n'

In [15]:
print(r("fit"))

try({fit})
$stl
Time Series:
Start = c(2016, 1) 
End = c(2018, 289) 
Frequency = 365 
         Data     Trend   Seasonal365     Remainder
2016.000  571  980.7551  -614.7600419   205.0049841
2016.003  681  981.1366  -539.1895173   239.0528828
2016.005  705  981.5182  -275.9675006    -0.5507106
2016.008 1111  981.8998    45.7783150    83.3218972
2016.011 1223  982.2814   107.0449368   133.6736988
2016.014 1249  982.6629    84.1171588   182.2199000
2016.016 1123  983.0445   -90.2038551   230.1593373
2016.019 1117  983.4261  -126.7878506   260.3617561
2016.022  868  983.8077   -51.6145697   -64.1931013
2016.025  937  984.1892   102.7729662  -149.9622139
2016.027 1244  984.5708   199.0595792    60.3695964
2016.030 1306  984.9524   341.1917752   -20.1441762
2016.033 1218  985.3340   123.7666990   108.8993233
2016.036 1118  985.7156    49.8891239    82.3953218
2016.038 1078  986.0971  -147.2457679   239.1486369
2016.041  858  986.4787   -99.9437109   -28.5349967
2016.044  861  986.8603   191.

In [16]:
r("fit_arima_fc_cb <- forecast(fit, h=442)")

'try({fit_arima_fc_cb <- forecast(fit, h=442)})\n'

In [17]:
print(r("(fit_arima_fc_cb)"))

try({(fit_arima_fc_cb)})
          Point Forecast       Lo 80     Hi 80      Lo 95    Hi 95
2018.7918      1572.3208 1269.554918 1875.0866 1109.28036 2035.361
2018.7945      1509.9277 1196.371399 1823.4841 1030.38469 1989.471
2018.7973      1418.9710 1104.991624 1732.9504  938.78097 1899.161
2018.8000      1302.9956  980.667239 1625.3239  810.03692 1795.954
2018.8027      1303.5072  978.206923 1628.8075  806.00335 1801.011
2018.8055      1151.9680  826.505263 1477.4307  654.21571 1649.720
2018.8082      1005.7511  676.574613 1334.9276  502.31910 1509.183
2018.8110      1275.3370  941.560852 1609.1132  764.87042 1785.804
2018.8137      1387.8637 1051.909791 1723.8176  874.06653 1901.661
2018.8164      1422.9421 1086.521510 1759.3626  908.43123 1937.453
2018.8192      1458.5591 1122.055034 1795.0632  943.92054 1973.198
2018.8219      2351.6830 2015.045348 2688.3206 1836.84015 2866.526
2018.8247      1124.3878  787.295197 1461.4804  608.84916 1639.926
2018.8274       992.0973  654.130070 

In [18]:
r("df_mape <- data.frame(rnb, fit_arima_fc_cb$fitted)")
r("mape_arima <- mean(abs((df_mape$rnb - df_mape$fit_arima_fc_cb.fitted)/df_mape$rnb))")
print(r("mape_arima"))

try({mape_arima})
[1] 0.1206377



In [19]:
r("df_rnb <- c(rnb, rep(NA,442))") 
r("df_arima <- c(fit_arima_fc_cb$fitted, fit_arima_fc_cb$mean) ")
#dfreal <- c(DF$y, rep(NA,h))
r("df_all_cb <- data.frame(df_rnb,df_arima)") 
r("names(df_all_cb) <- c('real','arima')") 

"try({names(df_all_cb) <- c('real','arima')})\n"

In [20]:
print(r("df_all_cb"))

try({df_all_cb})
     real     arima
1     571  569.8142
2     681  654.9122
3     705  885.1161
4    1111 1091.4673
5    1223 1205.7478
6    1249 1238.5185
7    1123 1075.5227
8    1117 1039.8924
9     868 1092.2898
10    937 1123.8638
11   1244 1226.2517
12   1306 1499.8868
13   1218 1268.4307
14   1118 1199.6774
15   1078  972.1310
16    858 1028.7376
17    861 1214.1112
18   1175 1226.9536
19   1331 1374.3818
20   1368 1363.1765
21   1437 1276.4932
22   1154 1026.1778
23    904 1057.8502
24    892 1054.3056
25   1236 1201.7847
26   1309 1355.2148
27   1308 1309.2921
28   1342 1140.0443
29   1270 1088.7763
30    914 1021.5536
31    866 1094.5143
32   1254 1171.1851
33   1436 1355.0274
34   1390 1411.8408
35   1287 1177.8382
36   1161  936.0933
37    940 1017.6515
38    803 1076.6914
39   1268 1248.2778
40   1461 1414.1166
41   1434 1487.0430
42   1454 1219.5263
43   1224 1064.6544
44    909 1001.4647
45    791 1080.5133
46   1313 1143.2033
47   1438 1375.5017
48   1434 1402.7932
49 

In [34]:
r("library(ggplot2)")
r("library(dplyr)")

r("df_all1 <- data.frame(dates=seq(from=(as.POSIXct(strftime('2016-01-01'))),length.out = nrow(df_all),by='days'),data = df_all)")
r("dfplot <- df_all1 %>% gather(key, value, -dates)") 

r("jpeg('real vs all models.jpg', height=4.25, width=5.5, res=200, units = 'in')")
r("p <- ggplot(dfplot, mapping = aes(x = dates, y = value, color = key) ) + geom_line() + ggtitle('real vs all models')") 
#r("return(print(p))")
print(r("p"))

try({p})
Error in try({ : object 'p' not found



In [21]:
r("write.csv(fit_arima_fc_cb, 'cb_arima_pred_fc.csv')")
r("write.csv(df_all_cb, 'cb_df_all_arima.csv')")

"try({write.csv(df_all_cb, 'cb_df_all_arima.csv')})\n"